# ✍️ Ad Copy Generation with OpenAI-style API via Hugging Face  
*Using DeepSeek-v3 hosted on Hugging Face to generate personalized product ads*

This notebook generates promotional ad copy for fashion products using a large language model (LLM) hosted on Hugging Face.  
We call an **OpenAI-compatible chat endpoint** powered by the model `deepseek/deepseek-v3-0324` to generate short, stylish ad lines.

We construct prompts using structured product information:
- `productDisplayName`
- `articleType`
- `baseColour`
- `season`

These ad copies are designed for:
- 🧵 E-commerce product pages
- 📢 Social media captions
- 📲 Push notifications and personalized ads

The final output is a curated dataset of clean, high-quality ad lines for each product in our demo set.


## 📦 Imports & Path Setup

This notebook uses `requests` to call the Hugging Face-hosted LLM via an OpenAI-style API.  
We also prepare relative file paths for loading product metadata and saving the generated ad copies.

In [ ]:
# Skip this cell if you're running in an environment where these are already installed.
# !pip install -q transformers
# !pip install -q huggingface_hub

In [17]:
# Imports
import os
import re
import sys
import pandas as pd
import requests

# Paths Setup
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..")) 
DATA_DIR = os.path.join(BASE_DIR, "data")

RAW_DIR = os.path.join(DATA_DIR, "raw")
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")

STYLES_CSV = os.path.join(RAW_DIR, "styles.csv")

AD_OUTPUT_CSV = os.path.join(PROCESSED_DIR, "sample_ad_copy_outputs.csv")

## 🗂️ Load and Prepare Product Metadata

We load a cleaned sample of fashion product metadata that contains all required fields for ad copy generation.

Each row must have:
- `productDisplayName` — the product’s title or name
- `articleType` — the category of clothing (e.g., Dress, Kurta, Top)
- `baseColour` — the primary color of the product
- `season` — the seasonal context of the item (e.g., Summer, Winter)

These fields are used to generate context-rich prompts for the language model.

In [6]:
# Load data
df = pd.read_csv(STYLES_CSV)

# Drop rows missing any required prompt fields
df = df.dropna(subset=["productDisplayName", "articleType", "baseColour", "season"])

print(f"✅ Loaded {len(df)} valid rows with complete product metadata.")
df.head()

✅ Loaded 41569 valid rows with complete product metadata.


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg


## ✉️ Prompt Construction for Ad Copy Generation

We construct a tailored prompt for each product using its:
- `productDisplayName` — to personalize the copy
- `articleType` — to define what kind of clothing it is
- `baseColour` — to guide visual emphasis
- `season` — to give seasonal context

Each prompt follows a fixed template and is sent to the language model to generate a short, natural-sounding promotional ad line.

In [7]:
# Define prompt template
def build_prompt(row):
    return (
        f"Write a short, engaging ad line for the following product.\n"
        f"Product: {row['productDisplayName']}.\n"
        f"Type: {row['articleType']}, Color: {row['baseColour']}, Season: {row['season']}.\n"
    )

# Apply template to each row
df["prompt"] = df.apply(build_prompt, axis=1)

# Preview a sample prompt
print(df["prompt"].iloc[0])

Write a short, engaging ad line for the following product.
Product: Turtle Check Men Navy Blue Shirt.
Type: Shirts, Color: Navy Blue, Season: Fall.



## 🤖 Setup API Call to DeepSeek-v3 Model via Hugging Face

We use the `deepseek-v3-0324` model hosted by Hugging Face to generate ad copy from structured prompts.

The model is accessed through an OpenAI-compatible chat endpoint (`/v3/openai/chat/completions`), and requires a valid **Hugging Face API token** for authentication.

>🔐 Your API key should be stored securely (e.g., in `.env`, GitHub Secrets, or environment variables).

In [9]:
# Set up Hugging Face API credentials
API_URL = "https://router.huggingface.co/novita/v3/openai/chat/completions"

headers = {
    "Authorization": "Bearer hf_", # Replace with your token
    "Content-Type": "application/json"
}

# Define a function to send a prompt and receive ad copy
def generate_ad(prompt, api_url=API_URL, headers=headers):
    payload = {
        "messages": [
            {
                "role": "user", 
                "content": prompt
            }
        ],
        "model": "deepseek/deepseek-v3-0324"
    }
    response = requests.post(api_url, headers=headers, json=payload)
    response.raise_for_status()  # Raises error if request failed
    return response.json()["choices"][0]["message"]["content"]

## 🧠 Generate Sample Ad Copy

To test our setup, we randomly select a few product prompts and pass them through the `deepseek-v3` API to generate short, engaging ad lines.

> These ad lines are meant to be catchy and relevant — ideal for use in e-commerce or digital advertising.

In [14]:
# Sample 5 product prompts
examples = df["prompt"].sample(5, random_state=42).tolist()
generated_ads = []

print("📢 Sample Ad Copy Results:\n")

for idx, prompt in enumerate(examples):
    ad = generate_ad(prompt)
    generated_ads.append(ad)
    
    print(f"Prompt:\n{prompt.strip()}\n")
    print(f"Ad Copy:\n{ad.strip()}\n{'-'*60}\n")

📢 Sample Ad Copy Results:

Prompt:
Write a short, engaging ad line for the following product.
Product: Angry Birds Women Pink T-shirt.
Type: Tshirts, Color: Pink, Season: Summer.

Ad Copy:
**"Stay fierce & fun this summer! 🎀💥 Rock the Angry Birds Women's Pink T-Shirt – bold, comfy, and totally you!"**  

*(Bonus: Add emojis like 🌞 or 🐦 for extra flair!)*
------------------------------------------------------------

Prompt:
Write a short, engaging ad line for the following product.
Product: Jockey Essence Black Bra 2004.
Type: Bra, Color: Black, Season: Winter.

Ad Copy:
**"Stay effortlessly chic this winter with the Jockey Essence Black Bra 2004—comfort meets elegance in every curve."**  

(Short, stylish, and highlights comfort + seasonal appeal!)
------------------------------------------------------------

Prompt:
Write a short, engaging ad line for the following product.
Product: ADIDAS Unisex New York Chronographic Digital Black Red Watch.
Type: Watches, Color: Black, Season: Wint

## 🧽 Clean and Extract Final Ad Copy

The model may return additional formatting such as:
- Markdown bold (`**"..."**`)
- Quotation marks (`"..."` or `'...'`)
- Multiple sentences or headings

We extract and clean the **core ad line** using a regex pattern.  
If the expected pattern isn't found, we fall back to the full model output.

In [15]:
cleaned_ads = []

for idx, ad in enumerate(generated_ads):
    ad = ad.strip()

    # Try to extract bold-quoted ad copy: **"..."**
    match = re.findall(r'\*\*"?([^*"]+)"?\*\*', ad)

    if match:
        cleaned = match[0]
    else:
        # Remove any extra quotes or markdown syntax
        cleaned = ad.replace('"', '').replace("'", '').strip()

    cleaned_ads.append(cleaned)
    print(f"{idx+1}. {cleaned}")

1. Stay fierce & fun this summer! 🎀💥 Rock the Angry Birds Women's Pink T-Shirt – bold, comfy, and totally you!
2. Stay effortlessly chic this winter with the Jockey Essence Black Bra 2004—comfort meets elegance in every curve.
3. Stay sharp this winter with the ADIDAS New York Chrono Watch—bold black style meets digital precision. Time never looked this cool! ⏱️🔥 #ADIDAS #WinterReady
4. Step into summer with Clarks Women's Balti Zing Beige Heels – effortless elegance for every sunny adventure!
5. Stay sharp this fall in crisp white style. ADIDAS Men's Graphic Tee—comfort meets cool. 🍂⚡


## 💾 Save Ad Copy Outputs to CSV

We attach each generated ad copy to its corresponding product and save the result as a CSV file.  
This dataset can be used in later stages such as:
- Campaign generation
- Dashboard display

In [ ]:
# Attach generated ad copy to sampled products
sample_df = df.sample(5, random_state=42).copy()
sample_df["ad_copy"] = cleaned_ads

# Save to CSV
sample_df[["id", "productDisplayName", "ad_copy"]].to_csv(AD_OUTPUT_CSV, index=False)
print(f"✅ Saved ad copy to → {AD_OUTPUT_CSV}")

## ✅ Summary

- Used product metadata (type, color, season, name) to construct structured prompts
- Generated ad copy using the `deepseek-v3` model via Hugging Face's OpenAI-style API
- Cleaned raw model outputs to extract high-quality, usable promotional lines
- Saved the generated ads for downstream use in OmniRetail AI’s promotion and campaign modules

➡️ Next: Use these ads in campaign exports or plug them into dashboards for content testing and personalization.